In [1]:
import requests
from bs4 import BeautifulSoup
import urllib
import re
from urlparse import urljoin
import pandas as pd
import html5lib

Good Resource for Beautiful soup: https://www.crummy.com/software/BeautifulSoup/bs4/doc/

In [32]:
base = 'http://isa.epfl.ch/imoniteur_ISAP/%21gedpublicreports.htm?ww_i_reportmodel=133685247'
r = requests.get(base)
soup = BeautifulSoup(r.text, 'html.parser')
frames = soup.findAll('frame')
for frame in frames:
    if(frame.attrs['name']=="toc"):
        href = urljoin(base,frame.attrs['src'])
        response = urllib.urlopen(href).read() 
        soup = BeautifulSoup(response, 'html.parser')
        
# Extract the name of parameters
tableFormat = soup.find("table", {"id": "format"})
tableFiltre = soup.find("table", {"id": "filtre"})

parameters = []

parameters.append(("ww_x_GPS","-1")) # need to find a way to extract this from the code
parameters.append(('ww_b_list','1'))
parameters.append(('ww_i_reportModel','133685247'))
# Get input value for format which text is html or excel hardcoded 
DATA_FORMAT = "html"
inputs = tableFormat.findAll("input")
for inp in inputs:
    if inp.text == DATA_FORMAT:
        parameters.append((inp.attrs['name'],inp.attrs['value']))
        #print inp.attrs['value']
        
resultsSelection = soup.find("td")
for selection in resultsSelection: 
    if selection.text == "Tous":
        parameters.append((selection.attrs['name'],""))
 
# Extract the name of parameters in tableFiltre
selects = tableFiltre.findAll('select')
for select in selects: 
    parameters.append((select.attrs['name'],""))
parameters


[('ww_x_GPS', '-1'),
 ('ww_b_list', '1'),
 ('ww_i_reportModel', '133685247'),
 (u'ww_i_reportModelXsl', u'133685270'),
 (u'ww_x_UNITE_ACAD', ''),
 (u'ww_x_PERIODE_ACAD', ''),
 (u'ww_x_PERIODE_PEDAGO', ''),
 (u'ww_x_HIVERETE', '')]

In [33]:
UNITE_ACAD = 'Informatique'
PERIODE_ACAD = '2016-2017'
PERIODE_PEDAGO = 'Bachelor semestre 1'

options = tableFiltre.findAll('option')

# Get input value for UNITE_ACAD which text is Informatique or excel hardcoded 
for option in options:
    if option.text == UNITE_ACAD :
        parameters[4] = (parameters[4][0],option.attrs['value'])

# Get input value for PERIODE_ACAD which text is Informatique or excel hardcoded 
    if option.text == PERIODE_ACAD:
        parameters[5] = (parameters[5][0], option.attrs['value'])

# Get input value for PERIODE_PEDAGO which text is Informatique or excel hardcoded 
    if option.text == PERIODE_PEDAGO:
         parameters[6] = (parameters[6][0], option.attrs['value'])

parameters

[('ww_x_GPS', '-1'),
 ('ww_b_list', '1'),
 ('ww_i_reportModel', '133685247'),
 (u'ww_i_reportModelXsl', u'133685270'),
 (u'ww_x_UNITE_ACAD', u'249847'),
 (u'ww_x_PERIODE_ACAD', u'355925344'),
 (u'ww_x_PERIODE_PEDAGO', u'249108'),
 (u'ww_x_HIVERETE', '')]

In [54]:
url = 'http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.html?'
rq = requests.get(url,parameters)
soup = BeautifulSoup(rq.text)
print(soup.prettify())

<html>
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <link href="gedpublicreports.css?ww_x_path=Gestac.Moniteur.Style" rel="stylesheet" type="text/css"/>
 </head>
 <body alink="#666666" bgcolor="#ffffff" link="#666666" marginheight="0" marginwidth="5" vlink="#666666">
  <fieldset style="text-align:right; width:40%; position:relative; margin-right: 10px;float:right; border: 0; padding: 0 0 8px 0;">
   <a href="!GEDREPORTS.html?ww_x_GPS=-1&amp;ww_b_list=1&amp;ww_i_reportModel=133685247&amp;ww_i_reportModelXsl=133685270&amp;ww_x_UNITE_ACAD=249847&amp;ww_x_PERIODE_ACAD=978181&amp;ww_x_PERIODE_PEDAGO=942175&amp;ww_x_HIVERETE=" style="color:#990033;">
    Identification pour accéder aux e-mails
    <br/>
    Login to access email adresses
   </a>
  </fieldset>
  <script>
   function mailList(x) {
   var vtop = (screen.height-200)/2;
   var vleft=(screen.width-600)/2;
   var w=open("", "emaillist", "Scrollbars=1,resizable=1,width=600,height=200,top="+vtop+",

In [61]:
table = soup.find('table')
dataframe = pd.read_html(rq.url)
dataframe = dataframe[0]
dataframe.columns = dataframe.ix[1]
dataframe = dataframe.ix[2:]
dataframe.columns

Index([            u'Civilité',           u'Nom Prénom',
       u'Orientation Bachelor',   u'Orientation Master',
             u'Spécialisation',         u'Filière opt.',
                     u'Mineur',               u'Statut',
               u'Type Echange',        u'Ecole Echange',
                  u'No Sciper',                     nan],
      dtype='object', name=1)

In [50]:
period_academique = []
period_academique_options = tableFiltre.find('select', {'name': parameters[5][0]}).find_all('option')
for option in period_academique_options:
    if option.text:
        period_academique.append((option.text,option.attrs['value']))

period_pedagogique = []
period_pedagogique_options = tableFiltre.find('select', {'name': parameters[6][0]}).find_all('option')
for option in period_pedagogique_options:
    if option.text:
        period_pedagogique.append((option.text,option.attrs['value']))
period_pedagogique

[(u'Bachelor semestre 1', u'249108'),
 (u'Bachelor semestre 2', u'249114'),
 (u'Bachelor semestre 3', u'942155'),
 (u'Bachelor semestre 4', u'942163'),
 (u'Bachelor semestre 5', u'942120'),
 (u'Bachelor semestre 5b', u'2226768'),
 (u'Bachelor semestre 6', u'942175'),
 (u'Bachelor semestre 6b', u'2226785'),
 (u'Master semestre 1', u'2230106'),
 (u'Master semestre 2', u'942192'),
 (u'Master semestre 3', u'2230128'),
 (u'Master semestre 4', u'2230140'),
 (u'Mineur semestre 1', u'2335667'),
 (u'Mineur semestre 2', u'2335676'),
 (u'Mise \xe0 niveau', u'2063602308'),
 (u'Projet Master automne', u'249127'),
 (u'Projet Master printemps', u'3781783'),
 (u'Semestre automne', u'953159'),
 (u'Semestre printemps', u'2754553'),
 (u'Stage automne 3\xe8me ann\xe9e', u'953137'),
 (u'Stage automne 4\xe8me ann\xe9e', u'2226616'),
 (u'Stage printemps 3\xe8me ann\xe9e', u'983606'),
 (u'Stage printemps 4\xe8me ann\xe9e', u'2226626'),
 (u'Stage printemps master', u'2227132')]

# Exercise 1

Let's extract data for all student registrations starting year 2007 for Bachelor Semester 1 and Bachelor Semester 6

In [40]:
parameters

[('ww_x_GPS', '-1'),
 ('ww_b_list', '1'),
 ('ww_i_reportModel', '133685247'),
 (u'ww_i_reportModelXsl', u'133685270'),
 (u'ww_x_UNITE_ACAD', u'249847'),
 (u'ww_x_PERIODE_ACAD', u'355925344'),
 (u'ww_x_PERIODE_PEDAGO', u'249108'),
 (u'ww_x_HIVERETE', '')]

In [190]:
# Search for Students with Bachelor Semester 1 and Bachelor Semester 6
bigDataFrame = pd.DataFrame()
dataframes = []
url = 'http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.html?'
period = [(semester, value) for (semester, value) in period_pedagogique if semester in 
          ['Bachelor semestre 1','Bachelor semestre 6']]
for (year,year_value) in period_academique:
    for (semester, semester_value) in period:
        parameters[5] = (parameters[5][0], year_value) # setting periode académique
        parameters[6] = (parameters[6][0], semester_value) # setting periode pédagogique
        rq = requests.get(url,parameters)
        dataframe = pd.read_html(rq.url)
        dataframe = dataframe[0]
        dataframe.columns = dataframe.ix[1]
        dataframe = dataframe.ix[2:]
        dataframe['Year'] = year.split('-')[0]
        dataframe['Semester']= semester
        dataframes.append(dataframe)

bigDataFrame = pd.concat(dataframes)
bigDataFrame = bigDataFrame[[u'Civilité',u'Nom Prénom',u'Statut',u'Year',u'No Sciper',u'Semester']]
bigDataFrame = bigDataFrame.reset_index(drop=True)

In [283]:
bigDataFrame.describe(include='all')

1,Civilité,Nom Prénom,Statut,Year,No Sciper,Semester
count,2383,2383,2383,2383,2383,2383
unique,2,1442,3,10,1442,2
top,Monsieur,Fol Charles,Présent,2014,181244,Bachelor semestre 1
freq,2178,5,2243,358,5,1719


In [120]:
bigDataFrame[u'No Sciper'].count()

2383

In [119]:
len(bigDataFrame[u'No Sciper'].unique())

1442

In [167]:
distinct_students = bigDataFrame.groupby(u'No Sciper')
distinct_students.describe()

1                 Civilité                                         Nom Prénom  \
No Sciper                                                                       
147008    count          2                                                  2   
          unique         1                                                  1   
          top     Monsieur                                        Good Xavier   
          freq           2                                                  2   
153762    count          1                                                  1   
          unique         1                                                  1   
          top     Monsieur                                       Conus Johann   
          freq           1                                                  1   
159516    count          1                                                  1   
          unique         1                                                  1   
          top     Monsieur                                        Raja Yanick   
          freq           1                                                  1   
159998    count          1                                                  1   
          unique         1                                                  1   
          top       Madame                                        Jesse Julia   
          freq           1                                                  1   
161091    count          1                                                  1   
          unique         1                                                  1   
          top       Madame                                   Grivet Ekaterina   
          freq           1                                                  1   
161127    count          1                                                  1   
          unique         1                                                  1   
          top     Monsieur                                      Sahy François   
          freq           1                                                  1   
161212    count          1                                                  1   
          unique         1                                                  1   
          top     Monsieur                                     Tamesna Rachid   
          freq           1                                                  1   
161220    count          2                                                  2   
          unique         1                                                  1   
...                    ...                                                ...   
275383    top     Monsieur                               Cochard Victor Angel   
          freq           1                                                  1   
275553    count          1                                                  1   
          unique         1                                                  1   
          top     Monsieur                                          Hu Julien   
          freq           1                                                  1   
275608    count          1                                                  1   
          unique         1                                                  1   
          top     Monsieur                                          Boujo Max   
          freq           1                                                  1   
276016    count          1                                                  1   
          unique         1                                                  1   
          top     Monsieur                            Wilders Eric Bruno John   
          freq           1                                                  1   
276323    count          1                                                  1   
          unique         1                                                  1   
          top     Monsieur     

In [137]:
bigDataFrame

1,Civilité,Nom Prénom,Statut,Year,No Sciper,Semester
0,Monsieur,Abbey Alexandre,Présent,2016-2017,235688,Bachelor semestre 1
1,Monsieur,Ahn Seongho,Présent,2016-2017,274015,Bachelor semestre 1
2,Madame,Alemanno Sara,Présent,2016-2017,268410,Bachelor semestre 1
3,Monsieur,Althaus Luca,Présent,2016-2017,271464,Bachelor semestre 1
4,Monsieur,Assi Karim,Présent,2016-2017,274518,Bachelor semestre 1
5,Monsieur,Badoux Luc-Antoine,Présent,2016-2017,249613,Bachelor semestre 1
6,Monsieur,Bagnoud Jérôme,Présent,2016-2017,262214,Bachelor semestre 1
7,Monsieur,Barbaras Yann Quentin,Présent,2016-2017,262239,Bachelor semestre 1
8,Monsieur,Barras Luca,Présent,2016-2017,257916,Bachelor semestre 1
9,Madame,Barsi Clémence Marie Sabine,Présent,2016-2017,271508,Bachelor semestre 1


In [291]:
def start(bigDataFrame):
    year = bigDataFrame.loc[bigDataFrame[u'No Sciper']==sciper].loc[bigDataFrame[u'Semester']=='Bachelor semestre 1'][u'Year']
    if year.any():
        year = int(year.iloc[0])
    else: 
        year = 'NaN'
    return year

def end(bigDataFrame):
    year = bigDataFrame.loc[bigDataFrame[u'No Sciper']==sciper].loc[bigDataFrame[u'Semester']=='Bachelor semestre 6'][u'Year']
    if year.any():
        year = int(year.iloc[-1])
    else: 
        year = 'NaN'
    return year

def duration(start,end):
    if start != 'NaN' and end != 'NaN':
        return (int(end)-int(start)+1)*12
    else: 
        return 'NaN'

df = [[sciper,start(bigDataFrame),end(bigDataFrame),duration(start(bigDataFrame),end(bigDataFrame))] for sciper in bigDataFrame[u'No Sciper'].unique()]   

In [294]:
dataframe = pd.DataFrame(df)
dataframe.columns = [u'No Sciper','Start','End','Duration']
dataframe = dataframe[dataframe['Duration']!='NaN']
#bigDataFrame.loc[bigDataFrame[u'No Sciper']=='251758']

dataframe
dataframe.sort_values(by=u'No Sciper',ascending=1)

,No Sciper,Start,End,Duration
1186,147008,2008,2010,36
1255,169569,2007,2009,36
1191,169731,2007,2010,48
1203,169795,2007,2009,36
1295,171195,2007,2009,36
1261,171619,2007,2009,36
1086,174905,2007,2010,48
1275,175190,2007,2009,36
1207,175280,2007,2009,36
1187,175379,2008,2010,36


In [302]:
# Partition the data between male and female students and compute average for male and female
def gender(sciper):
    return bigDataFrame[bigDataFrame[u'No Sciper']==sciper][u'Civilité'].iloc[0]
df = [[sciper,gender(sciper),start(bigDataFrame),end(bigDataFrame),duration(start(bigDataFrame),end(bigDataFrame))] 
      for sciper in bigDataFrame[u'No Sciper'].unique()]  

dataframe = pd.DataFrame(df)
dataframe.columns = [u'No Sciper',u'Gender','Start','End','Duration']
dataframe = dataframe[dataframe['Duration']!='NaN']
dataframe['Duration']=dataframe['Duration'].astype(int)
dataframe.groupby(u'Gender')['Duration'].mean()

Gender
Madame      36.000000
Monsieur    36.195652
Name: Duration, dtype: float64

# Exercise 2

In [130]:
# Search for Students with Bachelor Semester 1 and Bachelor Semester 6
masterDataFrame = pd.DataFrame()
dataframes = []
url = 'http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.html?'
period = [(semester, value) for (semester, value) in period_pedagogique if semester in 
          ['Master semestre 1','Master semestre 2','Master semester 3']]
for (year,year_value) in period_academique:
    for (semester, semester_value) in period:
        parameters[5] = (parameters[5][0], year_value) # setting periode académique
        parameters[6] = (parameters[6][0], semester_value) # setting periode pédagogique
        rq = requests.get(url,parameters)
        dataframe = pd.read_html(rq.url)
        dataframe = dataframe[0]
        dataframe.columns = dataframe.ix[1]
        dataframe = dataframe.ix[2:]
        dataframe['Year'] = year.split('-')[0]
        dataframe['Semester']= semester
        dataframes.append(dataframe)

masterDataFrame = pd.concat(dataframes)
masterDataFrame = masterDataFrame[[u'Civilité',u'Nom Prénom',u'Statut',u'Year',u'No Sciper',u'Semester',u'Spécialisation']]
masterDataFrame = masterDataFrame.reset_index(drop=True)

In [129]:
masterDataFrame

1,Civilité,Nom Prénom,Statut,Year,No Sciper,Semester,Spécialisation
0,Monsieur,Adams Marc Alain,Présent,2016-2017,271331,Master semestre 1,NaN
1,Monsieur,Adamsons Valdis,Présent,2016-2017,273101,Master semestre 1,NaN
2,Monsieur,Agapiou Stylianos,Présent,2016-2017,267543,Master semestre 1,NaN
3,Monsieur,Aiulfi Loris Sandro,Présent,2016-2017,202293,Master semestre 1,NaN
4,Monsieur,Alfonso Peterssen Alfonso,Présent,2016-2017,228219,Master semestre 1,Foundations of Software
5,Monsieur,Angerand Grégoire Georges Jacques,Présent,2016-2017,212464,Master semestre 1,NaN
6,Monsieur,Anongba Varela Dario Abougnan,Présent,2016-2017,226371,Master semestre 1,Software Systems
7,Monsieur,Aquil Mohammad Amimul Ihsan,Présent,2016-2017,273107,Master semestre 1,Foundations of Software
8,Monsieur,Bao Junze,Présent,2016-2017,266983,Master semestre 1,NaN
9,Monsieur,Barbera Antonino,Présent,2016-2017,273118,Master semestre 1,NaN
